# PTT文章抓取
<img src="ptt.jpg" width="40%" style="margin-left:0px;margin-top:15px;margin-bottom:15px;">
## 目標
1. 抓取PTT Movie版十頁文章
https://www.ptt.cc/bbs/movie/index.html
2. 把成果存成一個CSV檔案

## 重點
1. PTT有做最基本的檢查，檢查你的HTTP Header的部分使用的瀏覽器，所以你不能像以前什麼都不設定，不設定的情況就會發出403Forbidden的Error
2. 403是因為對面拒絕回答你，所以你應該做的是看看是為什麼被拒絕回答

## ✔ 開始

我們直接使用內建的urlopen，發現403
<img src="403.png" style="margin-left:0px;margin-top:15px;margin-bottom:15px;">



In [1]:
from urllib.request import urlopen
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
url = "https://www.ptt.cc/bbs/movie/index.html"
# 讀者可以自己把這行註解拿掉, 就會看到上圖的403
# urlopen(url)

## ✔ 確認Header

點開Chrome的 更多工具 -> 開發人員工具 -> Network -> 選到index.html
<img src="network.png"  style="margin-left:0px;margin-top:15px;margin-bottom:15px;margin-right:-5px;">


## ✔ 解決403 Forbidden

先利用內建的urllib.request的Request設計圖，創造出一個Request，並且填入User-Agent的部分

In [2]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
# BeautifulSoup每次會跳出要你選擇分析器的warning, 但我並不想固定住, 因為這樣放到別的作業系統會不能運作
# 而為了頁面的美化 不讓warning印出
import warnings
warnings.filterwarnings('ignore')

r = Request(url)
r.add_header("User-Agent", "Mozilla/5.0")
response = urlopen(r)
html = BeautifulSoup(response)
# 讀者可以把這行註解拿掉, 看看完整的html
# html

## 定義開網址函式

由於每次都要加上這個header，我們把它定義成一個函式吧

In [3]:
def send_ptt_request(url):
    r = Request(url)
    r.add_header("User-Agent", "Mozilla/5.0")
    response = urlopen(r)
    html = BeautifulSoup(response)
    return html

## 試試看一頁的擷取

寫程式一定要由小而大，先試試看一個頁面的擷取吧

In [4]:
url = "https://www.ptt.cc/bbs/movie/index.html"
# 用我們剛剛定義的函式來做
html = send_ptt_request(url)
# 得到每一篇文章的區域
articles = html.find_all("div", {"class":"r-ent"})
# 走過每一篇文章
for single_article in articles:
    # 得到title的超連結元素<a>
    title_area = single_article.find("div", {"class":"title"}).find("a")
    # 如果有title才繼續(被刪除的文章沒有title)
    if title_area:
        # 得到title的文字
        title = title_area.contents[0]
        # 得到title的超連結屬性href
        article_url = "https://www.ptt.cc" + title_area["href"]
        print(title, article_url)
        # [公告]我們不取
        if not "公告" in title:
            article_html = send_ptt_request(article_url)

            # 找到文章內容部分
            content = article_html.find("div", {"id":"main-content"})
            meta = content.find_all("div", {"class":"article-metaline"})
            
            # 由於文章內容都是混在一起的, 所以我們不能直接用find去找, 只好用扣掉的, 把我們不要的部分從元素裡扣掉(extract)
            # 扣掉 作者 標題 時間
            meta = content.find_all("div", {"class":"article-metaline"})
            for single_meta in meta:
                single_meta.extract()
                
            # 扣掉 看板名稱
            right_meta = content.find_all("div", {"class":"article-metaline-right"})
            for single_meta in right_meta:
                single_meta.extract()
            fs = content.find_all("span", {"class":"f2"})
            
            # 扣掉下方 來自 和 文章網址 的部分
            for single_f in fs:
                single_f.extract()
            pushes = content.find_all("div", {"class":"push"})
            
            # 扣掉 推文 的部分
            for single_push in pushes:
                single_push.extract()
                

[討論] 2018坎城影展頒獎與場刊分數 https://www.ptt.cc/bbs/movie/M.1526749272.A.895.html
[情報] iTunes Store 本週特價x4Kx新上架電影報 https://www.ptt.cc/bbs/movie/M.1526751626.A.048.html
Re: [討論] 海拉和金豹哪個王位取得比較沒異議？ https://www.ptt.cc/bbs/movie/M.1526752663.A.482.html
[請益] Re: [請益] 有人覺得死侍1難看但2好看的嗎 https://www.ptt.cc/bbs/movie/M.1526753789.A.504.html
[情報] 2018 坎城影展 得獎名單 https://www.ptt.cc/bbs/movie/M.1526753975.A.404.html
[有雷]  死侍2的彩蛋（算嗎？） https://www.ptt.cc/bbs/movie/M.1526755106.A.FA2.html
[新聞] 華人影片唯一希望 剩「江湖兒女」 https://www.ptt.cc/bbs/movie/M.1526755487.A.335.html
[情報] 2018坎城主競賽台灣片商代理情況 https://www.ptt.cc/bbs/movie/M.1526756080.A.8FD.html
[情報] 2018坎城影展最終場刊分數 https://www.ptt.cc/bbs/movie/M.1526760728.A.BFE.html
[片單] 警方無能求助無門、策畫私刑 的片單 https://www.ptt.cc/bbs/movie/M.1526761245.A.ED8.html
[新聞] Me Too+1！法國名導盧貝松被控下藥迷姦女 https://www.ptt.cc/bbs/movie/M.1526773074.A.5C6.html
[新聞] 日本是枝裕和奪金棕櫚　領獎「腿在發抖」 https://www.ptt.cc/bbs/movie/M.1526773183.A.F4F.html
[公告]《各式疑難雜症FAQ》 https://www.ptt.cc/bbs/movie/M.1391507188.A.EA3.html
[公告]

## 結合Pandas

先讓一頁做出一個DataFrame，之後我們才可以方便的輸出成csv或者sqlite

In [5]:
# import pandas
import pandas as pd
# 為了顯示的漂亮, 我刻意的把印出來的row只顯示10個和column只顯示十個, 大家練習的時候可以去掉下面兩行
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)

html = send_ptt_request(url)

# 把DataFrame的欄位準備好
df = pd.DataFrame(columns = ["title", "url", "content"])

articles = html.find_all("div", {"class":"r-ent"})
for single_article in articles:
    title_area = single_article.find("div", {"class":"title"}).find("a")
    if title_area:
        title = title_area.contents[0]
        article_url = "https://www.ptt.cc" + title_area["href"]
        if not "公告" in title:
            article_html = send_ptt_request(article_url)
            
            content = article_html.find("div", {"id":"main-content"})
            meta = content.find_all("div", {"class":"article-metaline"})
            
            for single_meta in meta:
                single_meta.extract()
            right_meta = content.find_all("div", {"class":"article-metaline-right"})
            
            for single_meta in right_meta:
                single_meta.extract()
            fs = content.find_all("span", {"class":"f2"})
            
            for single_f in fs:
                single_f.extract()
            pushes = content.find_all("div", {"class":"push"})
            
            for single_push in pushes:
                single_push.extract()
            # 針對每篇文章創出Series並且append到DataFrame裡
            s = pd.Series([title, article_url, content.text], index = ["title", "url", "content"])
            df = df.append(s, ignore_index = True)
df

,title,url,content
0,[討論] 2018坎城影展頒獎與場刊分數,https://www.ptt.cc/bbs/movie/M.1526749272.A.89...,\n\n今年坎城71，實在比坎城70精采多了!!!\n\n\n感覺每部競賽片都有一定水準，前...
1,[情報] iTunes Store 本週特價x4Kx新上架電影報,https://www.ptt.cc/bbs/movie/M.1526751626.A.04...,\n\n【本週 iTunes Store 特價電影 x 新片 x 4K 特價 】\n\n本週...
2,Re: [討論] 海拉和金豹哪個王位取得比較沒異議？,https://www.ptt.cc/bbs/movie/M.1526752663.A.48...,\n: 話說小魯之前看過雷神3和黑豹，\n: 這兩個故事有點類似，就是主角都是當地的領袖，然...
3,[請益] Re: [請益] 有人覺得死侍1難看但2好看的嗎,https://www.ptt.cc/bbs/movie/M.1526753789.A.50...,\n想借這篇問一下\n我也是覺得1難看\n但是難看的點是劇情跟流暢性\n好笑是滿好笑的\n很...
4,[情報] 2018 坎城影展 得獎名單,https://www.ptt.cc/bbs/movie/M.1526753975.A.40...,\n\n《小偷家族》｜是枝裕和｜日本\n\nhttps://i.imgur.com/s37m...
...,...,...,...
7,[情報] 2018坎城主競賽台灣片商代理情況,https://www.ptt.cc/bbs/movie/M.1526756080.A.8F...,\n想說整理一下，可能有錯，也有許多遺漏\n再麻煩大家補充指正謝謝\n有些是Rumor但可信...
8,[情報] 2018坎城影展最終場刊分數,https://www.ptt.cc/bbs/movie/M.1526760728.A.BF...,\nhttps://imgur.com/a/jlmJdN3\n\n基本上最後幾部的亮點就是N...
9,[片單] 警方無能求助無門、策畫私刑 的片單,https://www.ptt.cc/bbs/movie/M.1526761245.A.ED...,\n1. 請描述欲尋找電影的類型或特徵（請盡量描述清楚）\n\n標題下得不太好，如同以下舉例...
10,[新聞] Me Too+1！法國名導盧貝松被控下藥迷姦女,https://www.ptt.cc/bbs/movie/M.1526773074.A.5C...,\n新聞網址：https://tw.appledaily.com/new/realtime/...


In [6]:
html = send_ptt_request(url)
df = pd.DataFrame(columns = ["title", "url", "content"])
html = send_ptt_request(url)

# 使用range來做十次, times這個名字沒用到, 你可以用下底線_(不給名字)來取代
for times in range(0, 10):
    html = send_ptt_request(url)

    articles = html.find_all("div", {"class":"r-ent"})
    for single_article in articles:
        title_area = single_article.find("div", {"class":"title"}).find("a")
        if title_area:
            title = title_area.contents[0]
            article_url = "https://www.ptt.cc" + title_area["href"]
            if not "公告" in title:
                article_html = send_ptt_request(article_url)
                content = article_html.find("div", {"id":"main-content"})
                meta = content.find_all("div", {"class":"article-metaline"})

                for single_meta in meta:
                    single_meta.extract()
                right_meta = content.find_all("div", {"class":"article-metaline-right"})

                for single_meta in right_meta:
                    single_meta.extract()
                fs = content.find_all("span", {"class":"f2"})

                for single_f in fs:
                    single_f.extract()
                pushes = content.find_all("div", {"class":"push"})

                for single_push in pushes:
                    single_push.extract()

                s = pd.Series([title, article_url, content.text], index = ["title", "url", "content"])
                df = df.append(s, ignore_index = True)
    # 往下一頁前進, string參數可以找裡面文字符合我們帶入字串的元素
    url = "https://www.ptt.cc" + html.find("a", string="‹ 上頁")["href"]
df

,title,url,content
0,[討論] 2018坎城影展頒獎與場刊分數,https://www.ptt.cc/bbs/movie/M.1526749272.A.89...,\n\n今年坎城71，實在比坎城70精采多了!!!\n\n\n感覺每部競賽片都有一定水準，前...
1,[情報] iTunes Store 本週特價x4Kx新上架電影報,https://www.ptt.cc/bbs/movie/M.1526751626.A.04...,\n\n【本週 iTunes Store 特價電影 x 新片 x 4K 特價 】\n\n本週...
2,Re: [討論] 海拉和金豹哪個王位取得比較沒異議？,https://www.ptt.cc/bbs/movie/M.1526752663.A.48...,\n: 話說小魯之前看過雷神3和黑豹，\n: 這兩個故事有點類似，就是主角都是當地的領袖，然...
3,[請益] Re: [請益] 有人覺得死侍1難看但2好看的嗎,https://www.ptt.cc/bbs/movie/M.1526753789.A.50...,\n想借這篇問一下\n我也是覺得1難看\n但是難看的點是劇情跟流暢性\n好笑是滿好笑的\n很...
4,[情報] 2018 坎城影展 得獎名單,https://www.ptt.cc/bbs/movie/M.1526753975.A.40...,\n\n《小偷家族》｜是枝裕和｜日本\n\nhttps://i.imgur.com/s37m...
...,...,...,...
186,[討論] 有人檢舉過在戲院錄影的人嗎？,https://www.ptt.cc/bbs/movie/M.1526559101.A.1F...,\n去戲院看電影肯定對播放前宣導禁止錄影那個畫面不陌生\n畫面下方也會附上兩個檢舉電話\n我...
187,Re: [ 好雷] 死侍2的各種彩蛋討論,https://www.ptt.cc/bbs/movie/M.1526560681.A.C2...,\np幣少少 加減賺個\n\n\n防雷\n\n\n\n\n\n\n\n\n\n\n\n\n\...
188,[新聞] 劉若英「後來的我們」 台北電影節台灣首映,https://www.ptt.cc/bbs/movie/M.1526562610.A.69...,\n新聞網址：https://stars.udn.com/star/story/10090/...
189,[問片] 關於飛機上母親尋找女兒的電影,https://www.ptt.cc/bbs/movie/M.1526563123.A.93...,\n想問有部關於飛機上一個母親尋找失蹤的女兒，但全飛機上的乘客都覺得他在發瘋，在百\n般阻擾...


In [7]:
df.to_csv("ptt.csv", encoding="utf-8")